# TOPIQ: Top-down Image Quality Assessment Demo

This notebook demonstrates how to use the TOPIQ standalone implementation for image quality assessment tasks. TOPIQ (TOP-down approach for Image Quality assessment) is a state-of-the-art model that uses a cross-scale feature attention network to evaluate image quality.

## What is TOPIQ?

TOPIQ is a model described in the paper ["TOPIQ: A Top-down Approach from Semantics to Distortions for Image Quality Assessment"](https://arxiv.org/abs/2308.03060). It provides accurate image quality assessment using a unique cross-scale feature attention network approach.

The model offers multiple variants:
- No-reference (NR) assessment - evaluates quality without needing a reference image
- Full-reference (FR) assessment - compares a distorted image to a reference image
- Face-specific assessment - specialized for evaluating face image quality
- Aesthetic assessment - evaluates aesthetic quality rather than just technical quality

## Setup

First, let's install any required packages if needed and set up our environment.

In [ ]:
# Uncomment and run if you need to install these packages
# !pip install torch torchvision timm numpy Pillow requests matplotlib

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import requests
from io import BytesIO
import time
from pathlib import Path

# Import TOPIQ model
from topiq_model import create_topiq, load_image

# Set up device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Preparing Sample Images

Let's download and prepare a set of sample images with varying quality and content to test our model.

In [ ]:
# Create a directory for sample images
sample_dir = Path("sample_images")
sample_dir.mkdir(exist_ok=True)

# Function to download images
def download_image(url, save_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Downloaded {save_path}")
            return Image.open(save_path)
        else:
            print(f"Failed to download image, status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

In [ ]:
# Sample image URLs - these represent different quality levels and image types
image_urls = {
    "high_quality": "https://unsplash.com/photos/qrLV1ABFqFY/download?ixid=M3wxMjA3fDB8MXxzZWFyY2h8MTB8fGhpZ2glMjBxdWFsaXR5fGVufDB8fHx8MTcxMDgxMjUyN3ww&force=true&w=1920",
    "medium_quality": "https://unsplash.com/photos/random/download?ixid=M3wxMjA3fDB8MXxzZWFyY2h8MTF8fG1lZGl1bSUyMHF1YWxpdHl8ZW58MHx8fHwxNzEwODEyNjYzfDA&force=true&w=1080",
    "low_quality": "https://unsplash.com/photos/random/download?ixid=M3wxMjA3fDB8MXxzZWFyY2h8MTB8fGxvdyUyMHF1YWxpdHl8ZW58MHx8fHwxNzEwODEyNjkzfDA&force=true&w=640",
    "face": "https://unsplash.com/photos/random/download?ixid=M3wxMjA3fDB8MXxzZWFyY2h8Mnx8ZmFjZSUyMHBvcnRyYWl0fGVufDB8fHx8MTcxMDgxMjczMXww&force=true&w=1080",
    "artistic": "https://unsplash.com/photos/random/download?ixid=M3wxMjA3fDB8MXxzZWFyY2h8M3x8YXJ0aXN0aWMlMjBwaG90b2dyYXBoeXxlbnwwfHx8fDE3MTA4MTI3NTV8MA&force=true&w=1080"
}

# Dictionary to store our images
images = {}

# Download images if they don't already exist
for name, url in image_urls.items():
    img_path = sample_dir / f"{name}.jpg"
    if img_path.exists():
        print(f"Loading existing image: {img_path}")
        images[name] = Image.open(img_path)
    else:
        print(f"Downloading {name} image...")
        img = download_image(url, img_path)
        if img:
            images[name] = img

In [ ]:
# Display the images
plt.figure(figsize=(15, 10))

for i, (name, img) in enumerate(images.items()):
    plt.subplot(2, 3, i+1)
    plt.imshow(img)
    plt.title(name.replace('_', ' ').title())
    plt.axis('off')
    
plt.tight_layout()
plt.show()

## 1. No-Reference Image Quality Assessment

Let's start with no-reference image quality assessment, which evaluates the quality of images without requiring a reference image. This is useful for real-world scenarios where we don't have access to pristine reference images.

In [ ]:
# Load the TOPIQ No-Reference model
print("Loading TOPIQ No-Reference model...")
model_nr = create_topiq(model_name="topiq_nr", device=device)

# Function to assess image quality
def assess_quality(model, image_path):
    """Assess the quality of an image using a TOPIQ model"""
    start_time = time.time()
    
    # Load and process image
    if isinstance(image_path, Path):
        image_path = str(image_path)
    if isinstance(image_path, Image.Image):
        # Convert PIL image to tensor
        img = np.array(image_path).astype(np.float32) / 255.0
        img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)
    else:
        img = load_image(image_path).to(device)
    
    # Get quality score
    with torch.no_grad():
        score = model(img)
    
    end_time = time.time()
    processing_time = (end_time - start_time) * 1000  # ms
    
    return score.item(), processing_time

In [ ]:
# Assess quality of our sample images
results = {}
print("Assessing image quality using TOPIQ No-Reference model...")
print("-" * 60)
print(f"{'Image':<15} | {'Score':<10} | {'Time (ms)':<10}")
print("-" * 60)

for name, img in images.items():
    img_path = sample_dir / f"{name}.jpg"
    score, proc_time = assess_quality(model_nr, img_path)
    results[name] = score
    print(f"{name:<15} | {score:.4f}      | {proc_time:.2f}")

print("-" * 60)

### Visualizing No-Reference Quality Scores

Now let's visualize the quality scores for our sample images:

In [ ]:
plt.figure(figsize=(12, 6))

# Create bar chart of quality scores
names = list(results.keys())
scores = list(results.values())

plt.bar(names, scores, color='skyblue')
plt.xlabel('Images')
plt.ylabel('Quality Score (0-1)')
plt.title('TOPIQ No-Reference Quality Assessment')
plt.ylim(0, 1)

# Add score values on top of bars
for i, score in enumerate(scores):
    plt.text(i, score + 0.02, f'{score:.4f}', ha='center')

plt.tight_layout()
plt.show()

### Image Grid with Quality Scores

Let's also display the images alongside their quality scores:

In [ ]:
plt.figure(figsize=(15, 10))

for i, (name, img) in enumerate(images.items()):
    score = results[name]
    plt.subplot(2, 3, i+1)
    plt.imshow(img)
    plt.title(f"{name.replace('_', ' ').title()}\nQuality: {score:.4f}")
    plt.axis('off')
    
plt.tight_layout()
plt.show()

## 2. Face-specific Image Quality Assessment

TOPIQ provides specialized models for face image quality assessment. This is particularly useful for applications involving facial recognition, portrait photography, or facial analysis where the quality of face images is crucial.

Let's test the face-specific model on our face image and compare it with the general model:

In [ ]:
# Load the face-specific TOPIQ model
print("Loading TOPIQ Face Quality Assessment model...")
model_face = create_topiq(model_name="topiq_nr-face", device=device)

# Check if we have a face image in our collection
if 'face' in images:
    face_img_path = sample_dir / "face.jpg"
    
    # Get quality scores from both models
    face_score, face_time = assess_quality(model_face, face_img_path)
    general_score = results['face']  # We already computed this with the general model
    
    print(f"Face-specific quality score: {face_score:.4f} (processed in {face_time:.2f} ms)")
    print(f"General quality score:      {general_score:.4f}")
    
    # Display the face image with both scores
    plt.figure(figsize=(8, 8))
    plt.imshow(images['face'])
    plt.title(f"Face Image Quality Assessment\n"
              f"Face-specific score: {face_score:.4f}\n"
              f"General score: {general_score:.4f}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("No face image found in the dataset. Please add a face image to test this model.")

### Comparing Face Models

TOPIQ offers multiple variants of face quality assessment models. Let's compare them:

In [ ]:
# Compare different face models if available
if 'face' in images:
    face_img_path = sample_dir / "face.jpg"
    
    face_models = {
        "topiq_nr-face": create_topiq(model_name="topiq_nr-face", device=device),
        "topiq_nr-face-v1": create_topiq(model_name="topiq_nr-face-v1", device=device),
    }
    
    # Evaluate with each model
    face_results = {}
    print("\nComparing different face quality assessment models:")
    print("-" * 60)
    print(f"{'Model':<20} | {'Score':<10} | {'Time (ms)':<10}")
    print("-" * 60)
    
    for model_name, model in face_models.items():
        score, proc_time = assess_quality(model, face_img_path)
        face_results[model_name] = score
        print(f"{model_name:<20} | {score:.4f}      | {proc_time:.2f}")
        
    print("-" * 60)
    
    # Add the general model for comparison
    face_results["General NR"] = results['face']
    
    # Create a bar chart to compare models
    plt.figure(figsize=(10, 6))
    plt.bar(face_results.keys(), face_results.values(), color=['blue', 'green', 'orange'])
    plt.xlabel('Model')
    plt.ylabel('Quality Score (0-1)')
    plt.title('Face Quality Assessment Model Comparison')
    plt.ylim(0, 1)
    
    # Add score values on top of bars
    for i, (model, score) in enumerate(face_results.items()):
        plt.text(i, score + 0.02, f'{score:.4f}', ha='center')
    
    plt.tight_layout()
    plt.show()
else:
    print("No face image available for comparison.")

## 3. Aesthetic Image Quality Assessment

In addition to technical quality assessment, TOPIQ can also evaluate the aesthetic quality of images. Aesthetic quality refers to how visually pleasing or attractive an image is, rather than just its technical aspects.

The aesthetic model (topiq_iaa) returns scores on a scale from 1 to 10, where higher scores indicate better aesthetic quality.

In [ ]:
# Load the TOPIQ aesthetic model
print("Loading TOPIQ Aesthetic Assessment model...")
model_iaa = create_topiq(model_name="topiq_iaa", device=device)

# Assess aesthetic quality of our sample images
aesthetic_results = {}
print("Assessing aesthetic quality using TOPIQ IAA model...")
print("-" * 60)
print(f"{'Image':<15} | {'Aesthetic Score':<15} | {'Time (ms)':<10}")
print("-" * 60)

for name, img in images.items():
    img_path = sample_dir / f"{name}.jpg"
    score, proc_time = assess_quality(model_iaa, img_path)
    aesthetic_results[name] = score
    print(f"{name:<15} | {score:.2f}/10          | {proc_time:.2f}")

print("-" * 60)

In [ ]:
# Visualize aesthetic scores
plt.figure(figsize=(12, 6))

names = list(aesthetic_results.keys())
scores = list(aesthetic_results.values())

plt.bar(names, scores, color='purple')
plt.xlabel('Images')
plt.ylabel('Aesthetic Score (1-10)')
plt.title('TOPIQ Aesthetic Quality Assessment')
plt.ylim(1, 10)

# Add score values on top of bars
for i, score in enumerate(scores):
    plt.text(i, score + 0.2, f'{score:.2f}', ha='center')

plt.tight_layout()
plt.show()

### Comparing Technical vs. Aesthetic Quality

Let's compare technical quality scores (from the no-reference model) with aesthetic quality scores for our sample images. This can show interesting differences between technically good images and aesthetically pleasing ones.

In [ ]:
# Create a comparison between technical and aesthetic scores
plt.figure(figsize=(14, 7))

# Scale aesthetic scores to 0-1 for comparison
scaled_aesthetic_scores = [score / 10 for score in aesthetic_results.values()]

x = np.arange(len(names))
width = 0.35

# Create grouped bar chart
plt.bar(x - width/2, list(results.values()), width, label='Technical Quality (0-1)', color='skyblue')
plt.bar(x + width/2, scaled_aesthetic_scores, width, label='Aesthetic Quality (scaled to 0-1)', color='purple')

plt.xlabel('Images')
plt.ylabel('Score (0-1)')
plt.title('Technical vs. Aesthetic Quality Comparison')
plt.xticks(x, names)
plt.legend()

# Add score values on top of bars
for i, (tech_score, aes_score) in enumerate(zip(results.values(), aesthetic_results.values())):
    plt.text(i - width/2, tech_score + 0.02, f'{tech_score:.2f}', ha='center', va='bottom')
    plt.text(i + width/2, scaled_aesthetic_scores[i] + 0.02, f'{aes_score:.1f}/10', ha='center', va='bottom')

plt.tight_layout()
plt.show()

### Displaying Images with Technical and Aesthetic Scores

Let's visualize our images with both their technical and aesthetic quality scores:

In [ ]:
plt.figure(figsize=(15, 12))

for i, (name, img) in enumerate(images.items()):
    tech_score = results[name]
    aes_score = aesthetic_results[name]
    
    plt.subplot(2, 3, i+1)
    plt.imshow(img)
    plt.title(f"{name.replace('_', ' ').title()}\nTechnical: {tech_score:.2f}\nAesthetic: {aes_score:.2f}/10")
    plt.axis('off')
    
plt.tight_layout()
plt.show()

## 4. Full-Reference Image Quality Assessment

Full-Reference Image Quality Assessment (FR-IQA) measures the quality of an image by comparing it with a reference (undistorted) version. This approach can provide precise measurements of how much a processed or distorted image deviates from its original version.

Let's first create some distorted versions of our high-quality image to demonstrate FR-IQA:

In [ ]:
# Create distorted versions of our high-quality image
def create_distorted_images(reference_image_path, save_dir):
    # Open reference image
    ref_img = Image.open(reference_image_path)
    
    # Create distortions with different levels of JPEG compression
    compression_levels = [90, 60, 30, 10]  # JPEG quality levels
    distorted_imgs = {}
    
    for quality in compression_levels:
        dist_path = save_dir / f"distorted_q{quality}.jpg"
        if not dist_path.exists():
            print(f"Creating JPEG distortion with quality {quality}...")
            ref_img.save(dist_path, format="JPEG", quality=quality)
        
        dist_img = Image.open(dist_path)
        distorted_imgs[f"JPEG Q{quality}"] = dist_img
    
    return distorted_imgs

# Select a reference image (using high_quality)
reference_img_path = sample_dir / "high_quality.jpg"
if not reference_img_path.exists() and 'high_quality' in images:
    reference_img_path = sample_dir / "high_quality.jpg"

# Create distorted images
if reference_img_path.exists():
    print(f"Using {reference_img_path} as the reference image")
    distorted_images = create_distorted_images(reference_img_path, sample_dir)
else:
    print("No suitable reference image found. Please ensure a high-quality image is available.")

In [ ]:
# Display reference and distorted images
plt.figure(figsize=(15, 10))

# Display reference image
plt.subplot(2, 3, 1)
ref_img = Image.open(reference_img_path)
plt.imshow(ref_img)
plt.title("Reference Image")
plt.axis('off')

# Display distorted images
for i, (name, img) in enumerate(distorted_images.items()):
    plt.subplot(2, 3, i+2)
    plt.imshow(img)
    plt.title(name)
    plt.axis('off')

plt.tight_layout()
plt.show()

### Evaluating Full-Reference Quality

Now let's use TOPIQ's Full-Reference model to evaluate how closely each distorted image resembles the reference:

In [ ]:
# Load TOPIQ FR model
print("Loading TOPIQ Full-Reference model...")
model_fr = create_topiq(model_name="topiq_fr", device=device)

# Function for FR assessment
def assess_fr_quality(model, distorted_img_path, reference_img_path):
    """Assess the quality of a distorted image compared to a reference"""
    start_time = time.time()
    
    # Load images
    dist_img = load_image(distorted_img_path).to(device)
    ref_img = load_image(reference_img_path).to(device)
    
    # Get quality score
    with torch.no_grad():
        score = model(dist_img, ref_img)
    
    end_time = time.time()
    processing_time = (end_time - start_time) * 1000  # ms
    
    return score.item(), processing_time

In [ ]:
# Evaluate each distorted image
fr_results = {}
print("Evaluating distorted images using TOPIQ FR model...")
print("-" * 60)
print(f"{'Distortion':<15} | {'FR Score':<10} | {'Time (ms)':<10}")
print("-" * 60)

for name, img in distorted_images.items():
    quality = name.split("Q")[1]
    dist_path = sample_dir / f"distorted_q{quality}.jpg"
    score, proc_time = assess_fr_quality(model_fr, dist_path, reference_img_path)
    fr_results[name] = score
    print(f"{name:<15} | {score:.4f}      | {proc_time:.2f}")

print("-" * 60)

### Visualizing FR-IQA Results

Let's visualize the FR quality scores. We expect higher scores for images that are closer to the reference:

In [ ]:
# Create bar chart of FR scores
plt.figure(figsize=(10, 6))

names = list(fr_results.keys())
scores = list(fr_results.values())

plt.bar(names, scores, color='orange')
plt.xlabel('Distortion Level')
plt.ylabel('Quality Score (0-1)')
plt.title('TOPIQ Full-Reference Quality Assessment')
plt.ylim(0, 1)

# Add score values on top of bars
for i, score in enumerate(scores):
    plt.text(i, score + 0.02, f'{score:.4f}', ha='center')

plt.tight_layout()
plt.show()

### Displaying Images with FR Scores

Let's visualize the distorted images along with their FR quality scores:

In [ ]:
plt.figure(figsize=(15, 10))

# Display reference image
plt.subplot(2, 3, 1)
plt.imshow(ref_img)
plt.title("Reference Image")
plt.axis('off')

# Display distorted images with scores
for i, (name, img) in enumerate(distorted_images.items()):
    score = fr_results[name]
    plt.subplot(2, 3, i+2)
    plt.imshow(img)
    plt.title(f"{name}\nQuality: {score:.4f}")
    plt.axis('off')

plt.tight_layout()
plt.show()

## 5. Comparing Different TOPIQ Model Variants

TOPIQ offers several different model variants, each specialized for different tasks or trained on different datasets. Let's load several variants and compare how they evaluate the same set of images.

In [ ]:
# Load different TOPIQ model variants for comparison
print("Loading different TOPIQ model variants...")
topiq_models = {
    "General NR": create_topiq(model_name="topiq_nr", device=device),
    "Face NR": create_topiq(model_name="topiq_nr-face", device=device),
    "FLIVE NR": create_topiq(model_name="topiq_nr-flive", device=device),
    "SPAQ NR": create_topiq(model_name="topiq_nr-spaq", device=device),
    "Aesthetic": create_topiq(model_name="topiq_iaa", device=device)
}

In [ ]:
# Evaluate all images with all models
model_comparison = {model_name: {} for model_name in topiq_models.keys()}

print("Comparing TOPIQ models across all images...")
print("-" * 80)
print(f"{'Image':<15} | {'General NR':<10} | {'Face NR':<10} | {'FLIVE NR':<10} | {'SPAQ NR':<10} | {'Aesthetic':<10}")
print("-" * 80)

for img_name, img in images.items():
    img_path = sample_dir / f"{img_name}.jpg"
    scores = []
    
    # Evaluate image with each model
    for model_name, model in topiq_models.items():
        score, _ = assess_quality(model, img_path)
        model_comparison[model_name][img_name] = score
        
        # Format the score for display
        if model_name == "Aesthetic":
            formatted_score = f"{score:.2f}/10"
        else:
            formatted_score = f"{score:.4f}"
        
        scores.append(formatted_score)
    
    # Print results for this image
    print(f"{img_name:<15} | {scores[0]:<10} | {scores[1]:<10} | {scores[2]:<10} | {scores[3]:<10} | {scores[4]:<10}")

print("-" * 80)

### Visualizing Model Comparison with Radar Chart

Let's create a radar chart to better visualize how different models evaluate the same images:

In [ ]:
# Create a radar chart for comparing models
import matplotlib.pyplot as plt
import numpy as np

def radar_chart(data, categories, title):
    # Number of variables
    N = len(categories)
    
    # What will be the angle of each axis in the plot
    angles = [n / N * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Close the loop
    
    # Initialize the figure
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    # Draw one axis per variable and add labels
    plt.xticks(angles[:-1], categories, size=12)
    
    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([0.2, 0.4, 0.6, 0.8], ["0.2","0.4","0.6","0.8"], color="grey", size=10)
    plt.ylim(0, 1)
    
    # Plot data
    for i, (model_name, values) in enumerate(data.items()):
        # Scale aesthetic scores for comparison (1-10 to 0-1)
        if model_name == "Aesthetic":
            values = {k: v/10 for k, v in values.items()}
            
        # Plot values
        values_list = [values[cat] for cat in categories]
        values_list += values_list[:1]  # Close the loop
        
        ax.plot(angles, values_list, linewidth=2, linestyle='solid', label=model_name)
        ax.fill(angles, values_list, alpha=0.1)
    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    
    # Add title
    plt.title(title, size=15, y=1.1)
    
    return fig, ax

# Create the radar chart
radar_chart(model_comparison, list(images.keys()), "TOPIQ Model Comparison Across Images")
plt.tight_layout()
plt.show()

### Model Characteristics and Use Cases

Let's analyze the differences between these models:

- **General NR**: Balanced performance across various image types, good general-purpose model
- **Face NR**: Optimized for face image quality evaluation, considers face-specific attributes
- **FLIVE NR**: Trained on FLIVE dataset with more user-generated content, may better reflect user perceptions
- **SPAQ NR**: Trained on smartphone photos, optimized for mobile photography quality assessment
- **Aesthetic**: Evaluates artistic/aesthetic quality rather than just technical quality

The ideal model to use depends on your specific application:

## 6. Batch Processing for Efficiency

For applications that need to process many images, batch processing can significantly improve efficiency. Let's demonstrate how to process images in batches with TOPIQ:

In [ ]:
# Function to process images in batches
def batch_process(model, image_paths, batch_size=4):
    """Process multiple images in batches for efficiency"""
    all_scores = []
    total_images = len(image_paths)
    
    start_time = time.time()
    
    # Process in batches
    for i in range(0, total_images, batch_size):
        batch_paths = image_paths[i:min(i+batch_size, total_images)]
        batch_tensors = []
        
        # Load each image in the batch
        for path in batch_paths:
            tensor = load_image(path).to(device)
            batch_tensors.append(tensor)
        
        # Stack tensors into a batch
        batch_input = torch.cat(batch_tensors, dim=0)
        
        # Process batch
        with torch.no_grad():
            batch_scores = model(batch_input)
        
        # Add scores to results
        all_scores.extend(batch_scores.cpu().numpy().tolist())
    
    end_time = time.time()
    total_time = end_time - start_time
    
    return all_scores, total_time

In [ ]:
# Create a larger test set by duplicating our images
test_paths = [str(sample_dir / f"{name}.jpg") for name in images.keys()] * 5  # Repeat each image 5 times
print(f"Created test batch with {len(test_paths)} images")

# Test different batch sizes
model = topiq_models["General NR"]
batch_sizes = [1, 2, 4, 8, 16]  # Test different batch sizes
batch_results = {}

for bs in batch_sizes:
    if bs > len(test_paths):
        continue
    
    print(f"Processing with batch size {bs}...")
    scores, total_time = batch_process(model, test_paths, batch_size=bs)
    
    # Calculate metrics
    images_per_second = len(test_paths) / total_time
    ms_per_image = 1000 * total_time / len(test_paths)
    
    batch_results[bs] = {
        "total_time": total_time,
        "images_per_second": images_per_second,
        "ms_per_image": ms_per_image
    }
    
    print(f"  Processed {len(test_paths)} images in {total_time:.2f} seconds")
    print(f"  Speed: {images_per_second:.2f} images/sec ({ms_per_image:.2f} ms/image)")

In [ ]:
# Visualize batch processing efficiency
plt.figure(figsize=(12, 6))

# Plot processing speed (images per second)
plt.subplot(1, 2, 1)
plt.bar(
    [f"Batch {bs}" for bs in batch_results.keys()], 
    [data["images_per_second"] for data in batch_results.values()],
    color="green"
)
plt.ylabel("Images per second")
plt.title("Processing Speed by Batch Size")

# Plot time per image
plt.subplot(1, 2, 2)
plt.bar(
    [f"Batch {bs}" for bs in batch_results.keys()], 
    [data["ms_per_image"] for data in batch_results.values()],
    color="blue"
)
plt.ylabel("Time per image (ms)")
plt.title("Processing Time by Batch Size")

plt.tight_layout()
plt.show()

## Conclusion

In this notebook, we've explored the TOPIQ model's various capabilities:

1. **No-Reference Quality Assessment**: Evaluating image quality without a reference
2. **Face-specific Quality Assessment**: Specialized evaluation for face images
3. **Aesthetic Quality Assessment**: Measuring the artistic/visual appeal of images
4. **Full-Reference Quality Assessment**: Comparing distorted images to their references
5. **Model Comparison**: Understanding the differences between model variants
6. **Batch Processing**: Efficiently processing multiple images

TOPIQ provides a versatile toolkit for image quality assessment across various use cases, from technical quality evaluation to aesthetic judgment, and from general-purpose assessment to specialized domains like facial image analysis.

For more details on the TOPIQ model, refer to the [paper](https://arxiv.org/abs/2308.03060) and the [original implementation](https://github.com/chaofengc/IQA-PyTorch).